In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pre_processing import create_training_data,visualize_images
import random

/Users/egeozsoy/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/egeozsoy/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
learning_rate = 0.001
batch_size = 1024
num_input = 6400
num_classes = 2
dropout = 0.8 #prob to keep units
logs_path = 'logs/'

In [3]:
#network architecture params
output_1 = 32
output_2 = 64
output_3 = 1024
img_size = 80
conv_size = 5

In [4]:
X = tf.placeholder(tf.float32,[None,num_input])
Y = tf.placeholder(tf.float32,[None,num_classes])
keep_prob = tf.placeholder(tf.float32)#dropout

In [5]:
def conv_net(x,weights,biases,dropout):
    #because images are 80*80
    x = tf.reshape(x,shape=[-1,80,80,1])
    #stride = sliding, [1,?,?,1] dont change the ones
    conv1 = tf.nn.conv2d(x,weights['wc1'],strides=[1,1,1,1],padding='SAME')
    conv1 = tf.nn.bias_add(conv1,biases['bc1'])
    conv1 = tf.nn.relu(conv1) #maybe remove this
    conv1 = tf.nn.max_pool(conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    #maxpooling reduces image to half resulition because 2*2
    
    conv2 = tf.nn.conv2d(conv1,weights['wc2'],strides=[1,1,1,1],padding='SAME')
    conv2 = tf.nn.bias_add(conv2,biases['bc2'])
    conv2 = tf.nn.relu(conv2) #maybe remove this
    conv2 = tf.nn.max_pool(conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    
    fc1 = tf.reshape(conv2,[-1,weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1,weights['wd1']),biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1,dropout)
    
    out = tf.add(tf.matmul(fc1,weights['out']),biases['out'])
    
    return out

In [6]:
def chunks(images,labels, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(images), n):
        # Create an index range for l of n items:
        yield (images[i:i+n],labels[i:i+n])

In [7]:
def split_training_and_testing(images,labels,training_percantage=0.8):
    set_length = len(images)
    training_length = int(set_length * training_percantage)
    
    training_images = images[:training_length]
    training_labels = labels[:training_length]
    testing_images = images[training_length:]
    testing_labels = labels[training_length:]
    
    return training_images,training_labels,testing_images,testing_labels

In [8]:
def randomize_two_lists(images,labels):
    #shuffle lists but maintain order
    combined = list(zip(images, labels))
    random.shuffle(combined)
    images[:], labels[:] = zip(*combined)
    return images,labels

In [9]:
weights = {
    #5*5 conv with 1 input 32 outputs
    'wc1':tf.Variable(tf.random_normal([5,5,1,32])), #image res 80*80
    'wc2':tf.Variable(tf.random_normal([5,5,32,64])), #image res 40*40
    'wd1':tf.Variable(tf.random_normal([20*20*64,1024])), # image res 20*20
    'out':tf.Variable(tf.random_normal([1024,num_classes]))
}
biases = {
    'bc1':tf.Variable(tf.random_normal([32])),
    'bc2':tf.Variable(tf.random_normal([64])),
    'bd1':tf.Variable(tf.random_normal([1024])),
    'out':tf.Variable(tf.random_normal([num_classes]))
}

In [10]:
logits = conv_net(X,weights,biases,keep_prob)
prediction = tf.nn.softmax(logits)

In [11]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
tf.summary.scalar("loss",loss_op)

<tf.Tensor 'loss:0' shape=() dtype=string>

In [12]:
correct_pred = tf.equal(tf.argmax(prediction,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
tf.summary.scalar("accuracy",accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [13]:
merged_summary_op = tf.summary.merge_all()

In [14]:
images = np.load('images.npy')
labels = np.load('labels.npy')

In [15]:
images, labels = randomize_two_lists(images,labels)

In [16]:
training_images,training_labels,testing_images,testing_labels = split_training_and_testing(images,labels,training_percantage=0.1)

In [17]:
testing_images = testing_images.reshape(-1,6400)[0:10]
testing_labels = testing_labels[0:10]

In [18]:
model_name = 'model_{}_{}_{}_{}_{}_{}_{}'.format(learning_rate,dropout,output_1,output_2,output_3,img_size,conv_size)

In [19]:
init = tf.global_variables_initializer()

In [20]:
with tf.Session() as sess:
    sess.run(init)
    #for tensorboard
    summary_writer = tf.summary.FileWriter(logs_path+ model_name + '/', graph=tf.get_default_graph())
    for epoch in range(5):
        training_images,training_labels = randomize_two_lists(training_images,training_labels)
        for batch_x,batch_y in chunks(training_images,training_labels,batch_size):
            batch_x = batch_x.reshape(-1,6400)
            sess.run(train_op,feed_dict={X:batch_x,Y:batch_y,keep_prob:0.8})
            
        loss,acc,summary = sess.run([loss_op,accuracy,merged_summary_op],feed_dict={X:testing_images,Y:testing_labels,keep_prob:1.0})
        summary_writer.add_summary(summary, epoch)
        print("Completed {}".format(epoch), "Minibatch Loss= " + "{:.1f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
        
    print("Optimization Finished!")
        

Completed 0 Minibatch Loss= 36409268.0, Training Accuracy= 0.400


KeyboardInterrupt: 

In [ ]:
#add tensorboard and saving model